# Cable Robot Problem Generation
This code helps generate the coefficients to generate a cable robot planning problem, because it's tedious to do in C++.

In [ ]:
import numpy as np
import toppra as ta
import plotly.express as px
import pandas as pd
from pathlib import Path
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Defining the path and its derivatives

In [ ]:
# Let's create a simple rectangle
w, h = 1.5, 1
if False:
    path = np.array([[0, 0], [w, 0], [w, h], [0, h], [0, 0]]) + [1.5, 1.1] - [w/2, h/2]
    # Densely interpolate every dx units along the path
    dx = 0.05
    way_pts = []
    for s, e in zip(path[:-1], path[1:]):
        num_pts = int(np.linalg.norm(e - s) / dx)
        way_pts.append(np.linspace(s, e, num_pts))
    way_pts = np.concatenate(way_pts, axis=0)
elif False:
    t = np.linspace(0, 2*np.pi, 100)
    way_pts = 0.5 * np.vstack((np.cos(t), np.sin(t))).T + [1.5, 1.1]
elif True:
    th = np.linspace(0, 2*np.pi, 5, endpoint=False)
    # path = h * 0.5 * np.vstack((np.cos(th), np.sin(th))).T + [1.5, 1.1]
    path = h * 0.5 * np.vstack((-np.sin(th), np.cos(th))).T + [1.5, 1.1]
    path = np.roll(np.tile(path, (3, 1))[::3], 1, axis=0)
    path = np.vstack((path, path[0]))

    dx = 0.05
    way_pts = []
    for s, e in zip(path[:-1], path[1:]):
        num_pts = int(np.linalg.norm(e - s) / dx)
        way_pts.append(np.linspace(s, e, num_pts))
    way_pts = np.concatenate(way_pts, axis=0)

# Create a TOPPRA geometric path object, mainly just to get the spline interpolator
grid_pts = np.linspace(0, 10, way_pts.shape[0])
path = ta.SplineInterpolator(grid_pts, way_pts)

# And interpolate
t = np.linspace(0, 10, 1000)
x = path.eval(t)
xd = path.evald(t)
xdd = path.evaldd(t)
delta = 10 / 500

### Cable Robot Constraints & Objectives

In [ ]:
W_, H_ = 2.92, 2.32  # W/H of the frame
M = 1.0  # Mass of the end effector
g = np.array([0, -9.8])  # Gravity
# v_des = 0.15  # Desired velocity
# v_max = 0.30  # Maximum velocity
v_min = 0.02  # Minimum velocity
v_des = 1.00  # Desired velocity
v_max = 1.00  # Maximum velocity
t_m = 55  # Mid tension
t_min, t_max = 10, 100  # Minimum and maximum tension

In [ ]:
A = np.array([[W_, 0], [W_, H_], [0, H_], [0, 0]])  # 4 x 2
def wrench(x):  # x is n x 2
    ls = x[..., None] - A.T.reshape(1, 2, 4)  # n x 2 x 4
    return -ls / np.sqrt(np.sum(np.square(ls), axis=1, keepdims=True))  # n x 2 x 4
W = wrench(x)  # n x 2 x 4

In [ ]:
# Plot the robot & trajectory
fig = px.scatter(x=x[:, 0], y=x[:, 1]).update_traces(marker=dict(color='red', size=1))
fig.add_trace(px.line(x=A[[0,1,2,3,0], 0], y=A[[0,1,2,3,0], 1]).data[0])
# Axes equal
fig.update_layout(xaxis=dict(scaleanchor="y", scaleratio=1))

In [ ]:
# Desired speed is easy, just do ||x'^2.x - v_des^2||^2
# THIS IS ON X
xprimenorm2 = np.sum(np.square(xd), axis=1)
speed_quadratic_coeffs1d = np.stack([xprimenorm2**2, -2 * xprimenorm2 * v_des**2,
                                     np.ones_like(xprimenorm2) * v_des**4]).T
print(speed_quadratic_coeffs1d.shape)
speed_quadratic_coeffs1d[::100]

In [ ]:
# Same with speed limits
x_max = v_max**2 / np.sum(np.square(xd), axis=1)
speed_constr = np.stack([np.ones_like(x_max), x_max], axis=1)
speed_constr_zero = np.stack([-np.ones_like(x_max), np.zeros_like(x_max) - v_min], axis=1)
speed_constr = np.stack([speed_constr, speed_constr_zero], axis=1)
print(speed_constr.shape)
speed_constr[::100]

In [ ]:
# Acceleration limits, on u,x.  Since we assume point mass, we can ignore coriolis term B
# M.xdd + M.g - W.t = 0
# M.(x'.sdd + x''.sd^2) <= -M.g + W.tmax
# M.(x'.sdd + x''.sd^2) >= -M.g + W.tmin
# Actually I messed up, we need the feasible polygon, which I'm too lazy to do.
# Also in the cable robot I just use pseudoinverse anyway.  So let's do:
# Given desired wrench F=WT, compute T as:
#   T = T_mid + W^+.F
# T_min <= T_mid + W^+ (M.x'.sdd + Mx''.sd^2 + M.g) <= T_max
# T_min <= (W^+.M.x').sdd + (W^+.M.x'').sd^2 + (W^+.M.g + T_mid) <= T_max
# T_min <= (W^+.M.x').u + (W^+.M.x'').x + (W^+.M.g + T_mid) <= T_max
T_max = np.repeat(t_max, 4)
T_mid = np.repeat(t_m, 4)
T_min = np.repeat(t_min, 4)

W_pinv = np.linalg.pinv(W)
WiMxp = M * np.einsum('ijk,ik->ij', W_pinv, xd)
WiMxpp = M * np.einsum('ijk,ik->ij', W_pinv, xdd)
ff = M * np.einsum('ijk,k->ij', W_pinv, g)

accel_constr_max = np.stack([WiMxp, WiMxpp, T_max - T_mid - ff], axis=2)
accel_constr_min = np.stack([-WiMxp, -WiMxpp, -T_min + T_mid + ff], axis=2)
accel_constr = np.concatenate([accel_constr_max, accel_constr_min], axis=1)
print(accel_constr_max.shape, accel_constr_min.shape, accel_constr.shape)
# 200 x 8(num constraints) x 3 (2 variables)

# Remove redundant constraints
accel_constr_new = accel_constr
print(accel_constr_new[0])
print(accel_constr_new[0] / np.abs(accel_constr_new[0, :, 0:1]))
print(accel_constr_new[::100] / np.abs(accel_constr_new[::100, :, 0:1]))

In [ ]:
# Acceleration objectives, THIS IS ON U, X
# Minimize this: || (W^+.M.x').sdd + (W^+.M.x'').sd^2 + (W^+.M.g + T_mid) - T_mid ||^2
#                || a.u + b.x + c ||^2
#                a^2.u^2 + b^2.x^2 + 2ab.ux + 2ac.u + 2bc.x + c^2
a, b, c = WiMxp, WiMxpp, ff
dot_ = lambda x, y: np.einsum('ni,ni->n', x, y)
accel_quad = np.stack([dot_(a, a), dot_(b, b), 2*dot_(a, b), 2*dot_(a, c), 2*dot_(b, c), dot_(c,c)], axis=1)
print(accel_quad.shape)
print(accel_quad[10])
print(delta)

### Write out the constraints/objectives

Easiest way is just to write a literal c++ file.

In [ ]:
import numpy as np

def write_np_to_cpp(f, np_array, var_name):
    shape = np_array.shape
    dim = len(shape)

    if dim == 2:
        rows, cols = shape
        f.write(f"std::array<std::array<double, {cols}>, {rows}> {var_name} = {{{{\n")
        for row in np_array:
            row_str = ", ".join(map(str, row))
            f.write(f"    {{{{{row_str}}}}},\n")
        f.write("}};\n\n")

    elif dim == 3:
        depth, rows, cols = shape
        f.write(f"std::array<std::array<std::array<double, {cols}>, {rows}>, {depth}> {var_name} = {{{{\n")
        for matrix in np_array:
            f.write("    {{\n")
            for row in matrix:
                row_str = ", ".join(map(str, row))
                f.write(f"        {{{{{row_str}}}}},\n")
            f.write("    }},\n")
        f.write("}};\n\n")

    else:
        raise ValueError("Only 2D and 3D arrays are supported.")


In [ ]:
with open('data/CableRobotProblem.h', 'w') as f:
    f.write('#pragma once\n')
    f.write('#include <Eigen/Core>\n')
    f.write('#include <vector>\n')
    write_np_to_cpp(f, speed_constr, 'constr_x')
    write_np_to_cpp(f, accel_constr_new, 'constr_u')
    write_np_to_cpp(f, speed_quadratic_coeffs1d, 'quad_x')
    write_np_to_cpp(f, accel_quad, 'quad_u')

### Run the C++ code, now parse the results

In [ ]:
!cd ../../../build/ && ninja -j8 CableRobotExample.run

In [ ]:
root = Path('../../../build/gtsam_unstable/retiming/examples')
result = pd.read_csv(root / 'cable_robot_qopp_solution.csv', header=None, sep=',')[:-1]
result = result.rename(columns={0: 'x', 1: 'u'})
result_topp = pd.read_csv(root / 'cable_robot_topp_solution.csv', header=None, sep=',')[:-1]
result_topp = result_topp.rename(columns={0: 'x', 1: 'u'})
;

In [ ]:
traj = ta.parametrizer.ParametrizeSpline(path, t, np.sqrt(result['x'].values))
traj_topp = ta.parametrizer.ParametrizeSpline(path, t, np.sqrt(result_topp['x'].values))

In [ ]:
t_vals = np.linspace(0, traj.get_duration(), 500)
out = traj.eval(t_vals)
outd = traj.evald(t_vals)
outdd = traj.evaldd(t_vals)

t_vals_topp = np.linspace(0, traj_topp.get_duration(), 500)
out_topp = traj_topp.eval(t_vals_topp)
outd_topp = traj_topp.evald(t_vals_topp)
outdd_topp = traj_topp.evaldd(t_vals_topp)

In [ ]:
t1 = np.concatenate((traj.ss_waypoints, [traj.ss_waypoints[-1]]))
result['t'] = traj.ss_waypoints
result_topp['t'] = traj_topp.ss_waypoints

In [ ]:
# Cable Tensions
# T = (W^+.M.x').u + (W^+.M.x'').x + (W^+.M.g + T_mid)
# T_max = np.repeat(t_max, 4)
# T_mid = np.repeat(t_m, 4)
# T_min = np.repeat(t_min, 4)

# W_pinv = np.linalg.pinv(W)
# WiMxp = M * np.einsum('ijk,ik->ij', W_pinv, xd)
# WiMxpp = M * np.einsum('ijk,ik->ij', W_pinv, xdd)
# ff = M * np.einsum('ijk,k->ij', W_pinv, g)

Tensions = WiMxp * result['u'].values[:, None] + WiMxpp * result['x'].values[:, None] + ff + T_mid
Tensions_topp = WiMxp * result_topp['u'].values[:, None] + WiMxpp * result_topp['x'].values[:, None] + ff + T_mid

fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                    subplot_titles=("QOPP", "TOPP"))
for l in px.line(Tensions).data:
    fig.add_trace(l, row=1, col=1)
for l in px.line(Tensions_topp).data:
    fig.add_trace(l, row=1, col=2)
fig.show()

In [ ]:
# Matplotlib config
# Much slower, set to True when generating final plots
plt.rcParams['text.usetex'] = True
plt.rcParams['figure.figsize'] = [3.0, 2.0] # [6.4, 4.8]
# Set all label sizes to 8pt
plt.rcParams['font.size'] = 8
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['axes.titlesize'] = 9
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 6
plt.rcParams['figure.titlesize'] = 10
# Set line widths
plt.rcParams['axes.linewidth'] = 0.3
plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams['lines.markersize'] = 4
plt.rcParams['lines.markeredgewidth'] = 0.5
plt.rcParams['patch.linewidth'] = 0.3
plt.rcParams['grid.linewidth'] = 0.3
plt.rcParams['xtick.major.width'] = 0.3
plt.rcParams['ytick.major.width'] = 0.3
plt.rcParams['xtick.minor.width'] = 0.3
plt.rcParams['ytick.minor.width'] = 0.3
plt.rcParams['xtick.major.size'] = 2
plt.rcParams['ytick.major.size'] = 2
plt.rcParams['xtick.minor.size'] = 1
plt.rcParams['ytick.minor.size'] = 1

In [ ]:
# Publication-quality tension vs time
# Create a figure and a 2x1 grid of subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 2))

colors = ['k', 'r', 'g', 'b']
# Plotting Tensions on the first subplot
for i in range(4):
    axes[0].plot(result['t'], Tensions[:, i], colors[i], label=f'Cable {i+1} Tension', linewidth=1)
axes[0].plot(result['t'], t_max * np.ones_like(result['t']), 'k:', label=f'Tenison Limits', linewidth=1)
axes[0].plot(result['t'], t_min * np.ones_like(result['t']), 'k:', linewidth=1)
axes[0].set_xlim([0, result['t'].max() * 1.05])
axes[0].set_title('QOPP Solution (Ours)')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Cable Tensions (N)')
axes[0].legend(fontsize=4)

# Plotting Tensions_topp on the second subplot
for i in range(4):
    axes[1].plot(result_topp['t'], Tensions_topp[:, i], colors[i], label=f'Cable {i+1} Tension', linewidth=1)
axes[1].plot(result_topp['t'], t_max * np.ones_like(result_topp['t']), 'k:', label=f'Tenison Limits', linewidth=1)
axes[1].plot(result_topp['t'], t_min * np.ones_like(result_topp['t']), 'k:', linewidth=1)
axes[1].set_xlim([0, result_topp['t'].max() * 1.05])
axes[1].set_title('TOPP Solution')
axes[1].set_xlabel('Time (s)')
# axes[1].set_ylabel('Cable Tensions (N)')
# axes[1].legend()

# share y axis
axes[0].get_shared_y_axes().join(axes[0], axes[1])

# Adjust the layout
plt.tight_layout()

# Save the figure
# plt.savefig('/Users/gerry/DropboxGatech/Apps/Overleaf/ICRA2024_TrajectoryRetimingQuadraticObjectives/figs/TensionsCompare.eps')

# Show the plot
plt.show()


In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                    subplot_titles=("QOPP", "TOPP")).update_layout(height=300)
for l in px.scatter(result[:-1], y=['x', 'u']).update_traces(marker=dict(size=2), line=dict(width=1), mode='lines+markers').data:
    fig.add_trace(l, row=1, col=1)
for l in px.scatter(result_topp[:-1], y=['x', 'u']).update_traces(marker=dict(size=2), line=dict(width=1), mode='lines+markers').data:
    fig.add_trace(l, row=1, col=2)
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=("Position", "Position", "Velocity", "Velocity", "Acceleration", "Acceleration"))

# Add traces with specified colors
fig.add_trace(go.Scatter(x=t_vals, y=out[:, 0], mode='lines', name='Position X', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vals, y=out[:, 1], mode='lines', name='Position Y', line=dict(color='green')), row=1, col=1)

fig.add_trace(go.Scatter(x=t_vals, y=outd[:, 0], mode='lines', name='Velocity X', line=dict(color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=t_vals, y=outd[:, 1], mode='lines', name='Velocity Y', line=dict(color='green')), row=2, col=1)

fig.add_trace(go.Scatter(x=t_vals, y=outdd[:, 0], mode='lines', name='Acceleration X', line=dict(color='red')), row=3, col=1)
fig.add_trace(go.Scatter(x=t_vals, y=outdd[:, 1], mode='lines', name='Acceleration Y', line=dict(color='green')), row=3, col=1)

# Add traces with specified colors
fig.add_trace(go.Scatter(x=t_vals_topp, y=out_topp[:, 0], mode='lines', name='Position X', line=dict(dash='solid', color='darkred')), row=1, col=2)
fig.add_trace(go.Scatter(x=t_vals_topp, y=out_topp[:, 1], mode='lines', name='Position Y', line=dict(dash='solid', color='darkgreen')), row=1, col=2)

fig.add_trace(go.Scatter(x=t_vals_topp, y=outd_topp[:, 0], mode='lines', name='Velocity X', line=dict(dash='solid', color='darkred')), row=2, col=2)
fig.add_trace(go.Scatter(x=t_vals_topp, y=outd_topp[:, 1], mode='lines', name='Velocity Y', line=dict(dash='solid', color='darkgreen')), row=2, col=2)

fig.add_trace(go.Scatter(x=t_vals_topp, y=outdd_topp[:, 0], mode='lines', name='Acceleration X', line=dict(dash='solid', color='darkred')), row=3, col=2)
fig.add_trace(go.Scatter(x=t_vals_topp, y=outdd_topp[:, 1], mode='lines', name='Acceleration Y', line=dict(dash='solid', color='darkgreen')), row=3, col=2)

# Update layout
fig.update_layout(title_text="Stacked Plots of Position, Velocity, and Acceleration")
fig.update_layout(width=900, height=500)
fig.show()

In [ ]:
fig = px.scatter(x=traj(t_vals)[:, 0], y=traj(t_vals)[:, 1], opacity=0.2)
fig.update_traces(marker=dict(size=4))
fig.update_layout(xaxis=dict(scaleanchor="y", scaleratio=1))
fig.show()

### Export the path to the cable robot

Check the art_skills repo, `gerry06_toppra_send.ipynb`

In [ ]:
import pickle
with open('/tmp/star.pickle', 'wb') as f:
    pickle.dump(traj, f)

In [ ]:
with open('/tmp/star.pickle', 'rb') as f:
    print(pickle.load(f).cspl)